In [3]:
pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.6 MB 1.2 MB/s eta 0:00:34
   ---------------------------------------- 0.2/38.6 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.4/38.6 MB 2.8 MB/s eta 0:00:14
   - -------------------------------------- 1.1/38.6 MB 5.2 MB/s eta 0:00:08
   - -------------------------------------- 1.8/38.6 MB 7.7 MB/s eta 0:00:05
   -- ------------------------------------- 2.4/38.6 MB 8.8 MB/s eta 0:00:05
   --- ------------------------------------ 3.9/38.6 MB 12.3 MB/s eta 0:00:03
   ----- ---------------------------------- 5.3/38.6 MB 14.8 MB/s eta 0:00:03
   ------- ----------

In [1]:
import time
import openai
from openai import OpenAI
client = OpenAI(api_key = 'cxzczcz')

def generate_ai_images(payload):
    try:
        # Make a request to the DALL-E 3 API
        response = client.images.generate(
            model="dall-e-3", prompt=payload, size="1024x1024", n=1
        )
        # Assuming the API returns the generated image URL
        return {"image_url": response.data[0].url}
    except Exception as e:
        return {"error": str(e)}

def generate_ai_text(payload):
    try:
        prompt = (payload)
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        return {"error": str(e)}

In [2]:
def generate_logo_prompt(user_input):

    prompt = f"Extract keywords from the following text useful for logo generation: '{user_input}'. Avoid undesirable or irrelevant keywords which you think will add text or weird in the logo"
    placeholder = generate_ai_text(prompt)  
    prompts = [
        f"No text,Line art logo of {placeholder}, minimalistic, solid white background. Avoid Text and multiple images, background has to be white",
        f"No text,Flat vector logo of {placeholder},minimalistic, solid white background.Avoid Text and multiple images,background has to be white"
    ]
    selected_prompt = random.choice(prompts)

    return selected_prompt

In [3]:
import cv2
import numpy as np
import random
from PIL import Image, ImageDraw, ImageFont

def add_text_below_logo(image, text, slogan):
    def find_edges_and_crop(image_cv, offset=10):
        gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, threshold1=100, threshold2=200)
        y_indices, x_indices = np.where(edges > 0)
        top_edge, bottom_edge = np.min(y_indices), np.max(y_indices)
        left_edge, right_edge = np.min(x_indices), np.max(x_indices)
        top_edge = max(top_edge - offset, 0)
        bottom_edge = min(bottom_edge + offset, image_cv.shape[0])
        left_edge = max(left_edge - offset, 0)
        right_edge = min(right_edge + offset, image_cv.shape[1])
        cropped_image = image_cv[top_edge:bottom_edge, left_edge:right_edge]
        return cropped_image

    def calculate_font_size_and_canvas_width(draw, text, start_size, max_width, font_path):
        font_size = start_size
        font = ImageFont.truetype(font_path, font_size)
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        while text_width > .8 * max_width and font_size > 1:
            font_size -= 1
            font = ImageFont.truetype(font_path, font_size)
            text_bbox = draw.textbbox((0, 0), text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        return font, font_size, text_width, text_height

    def find_darkest_color(image_cv):
        colors, counts = np.unique(image_cv.reshape(-1, 3), axis=0, return_counts=True)
        darkest_color = min(colors, key=lambda col: col[0] + col[1] + col[2])
        return tuple(int(c) for c in darkest_color)

    # Convert PIL Image to OpenCV format
    image_cv = np.array(image)
    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)

    cropped_logo_cv = find_edges_and_crop(image_cv, offset=20)
    cropped_logo_pil = Image.fromarray(cv2.cvtColor(cropped_logo_cv, cv2.COLOR_BGR2RGB))

    # Find the darkest color in the logo
    text_color = find_darkest_color(cropped_logo_cv)

# #     Specify the path to your local font files
#     text_font = ["C:/Users/Abdul Wahab/ICMS Brochure/Fonts/CinzelDecorative-Bold.ttf",
#                      "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Montserrat-Bold.ttf",
#                         "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Helvetica CE Bold.otf"] 
#     slogan_font = ["C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Montserrat-Italic.ttf",
#                        "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Catamaran-Regular.ttf",
#                       "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Helvetica CE Medium.otf"]  

#     text_font_path = random.choice(text_font)
#     slogan_font_path = random.choice(slogan_font)
    
    text_font_path = "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Montserrat-Bold.ttf"
    slogan_font_path = "C:/Users/Abdul Wahab/ICMS Brochure/Fonts/Montserrat-Italic.ttf"
    
    # Initialize temporary canvas for text size calculation
    temp_canvas = Image.new('RGB', (400, 200), "white")
    draw = ImageDraw.Draw(temp_canvas)

    # Calculate font sizes for text and slogan
    main_text_font, _, main_text_width, main_text_height = calculate_font_size_and_canvas_width(draw, text, start_size=40, max_width=400 - 20, font_path=text_font_path)
    slogan_start_size = int(main_text_height // 1.5)
    slogan_font, _, slogan_width, slogan_height = calculate_font_size_and_canvas_width(draw, slogan, start_size=slogan_start_size, max_width=400 - 20, font_path=slogan_font_path)

    # Calculate canvas size
    canvas_width = 400
    max_logo_height = 200 * 0.5
    logo_aspect_ratio = cropped_logo_pil.width / cropped_logo_pil.height
    logo_height = int(min(max_logo_height, canvas_width / logo_aspect_ratio))
    logo_width = int(logo_height * logo_aspect_ratio)

    # Calculate total height needed for text, slogan, and logo
    total_height_needed = logo_height + main_text_height + slogan_height + 30
    canvas_height = max(200, total_height_needed)

    # Create new canvas with adjusted size
    new_canvas = Image.new('RGB', (canvas_width, canvas_height), "white")
    draw = ImageDraw.Draw(new_canvas)

    # Paste resized logo
    resized_logo = cropped_logo_pil.resize((logo_width, logo_height), Image.ANTIALIAS)
    logo_x = (canvas_width - logo_width) // 2
    logo_y = 10
    new_canvas.paste(resized_logo, (logo_x, logo_y))

    # Draw main text
    main_text_x = (canvas_width - main_text_width) //2
    new_canvas.paste(resized_logo, (logo_x, logo_y))

    # Draw main text
    main_text_x = (canvas_width - main_text_width) // 2
    main_text_y = logo_y + logo_height + 10  # Padding below logo
    draw.text((main_text_x, main_text_y), text, font=main_text_font, fill=text_color)

    # Draw slogan
    slogan_x = (canvas_width - slogan_width) // 2
    slogan_y = main_text_y + main_text_height + 5  # Padding below main text
    draw.text((slogan_x, slogan_y), slogan, font=slogan_font, fill=text_color)

    return new_canvas


In [4]:
import requests
from io import BytesIO
from PIL import Image
from rembg import remove 

def generate_logo(payload):
    try:
        project_details = payload
        project_name = project_details.get("Project_name", "")
        prompt = project_details.get("Prompt", "")
        slogan = project_details.get("Slogan","")

        logo_prompt = generate_logo_prompt(prompt)
        image_info = generate_ai_images(logo_prompt)

        if 'image_url' not in image_info:
            raise ValueError("No image URL found in the response from generate_ai_images")

        response = requests.get(image_info["image_url"])
        image = Image.open(BytesIO(response.content))
#         # Processing the image 
#         input = Image.open(input_path) 

#         # Removing the background from the given Image 
#         image = remove(image)

        final_logo = add_text_below_logo(image, project_name,slogan.capitalize())

        # Save or return the final logo
        final_logo_path = "C:/Users/Abdul Wahab/ICMS Brochure/Generated_images/gen_logo.png"
        final_logo.save(final_logo_path)
        return final_logo_path
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



C:\Users\Abdul Wahab\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [7]:
payload = {
    "Project_name": "TCG CLIFF GARDEN",
    "Slogan": "Crating better communities",
    "Prompt": "2 leaf showing serenity green colour"
}

In [8]:
# payload = {
#     "Project_name": "Kotle Patil",
#     "Slogan": "Creation not Construction",
#     "Prompt": "3 birds in a nest, ochre colour"
# }

In [10]:
import time
# Record the start time
start_time = time.time()

generate_logo(payload)

# Record the end time
end_time = time.time()

# Calculate the generation time
generation_time = end_time - start_time
print(f"Image generation time: {generation_time} seconds")

Image generation time: 19.458934783935547 seconds


C:\Users\Abdul Wahab\AppData\Local\Temp\ipykernel_39884\42585956.py:35: RuntimeWarning: overflow encountered in scalar add
  darkest_color = min(colors, key=lambda col: col[0] + col[1] + col[2])
C:\Users\Abdul Wahab\AppData\Local\Temp\ipykernel_39884\42585956.py:87: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_logo = cropped_logo_pil.resize((logo_width, logo_height), Image.ANTIALIAS)


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def logo_processor(image_path):
    """
    Process the logo image: load, convert to grayscale, find edges, crop, and return as PIL Image.
    """
    image_cv = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image_cv, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    y_indices, x_indices = np.where(edges > 0)
    if y_indices.size and x_indices.size:
        top_edge, bottom_edge = np.min(y_indices), np.max(y_indices)
        left_edge, right_edge = np.min(x_indices), np.max(x_indices)
        cropped_image_cv = image_cv[max(top_edge - 10, 0):min(bottom_edge + 10, image_cv.shape[0]),
                                    max(left_edge - 10, 0):min(right_edge + 10, image_cv.shape[1])]
        return Image.fromarray(cv2.cvtColor(cropped_image_cv, cv2.COLOR_BGR2RGB))
    else:
        return Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))
    
    
def calculate_font_size(text, font_path, max_width):
    """
    Calculate the appropriate font size for text to fit within a specified width.
    """
    font_size = 40
    font = ImageFont.truetype(font_path, font_size)
    text_width, _ = font.getsize(text)
    while text_width > max_width and font_size > 1:
        font_size -= 1
        font = ImageFont.truetype(font_path, font_size)
        text_width, _ = font.getsize(text)
    return font


def find_darkest_color(image):
    """
    Find the darkest color in the image for setting text color.
    """
    image_cv = np.array(image)
    colors, counts = np.unique(image_cv.reshape(-1, 3), axis=0, return_counts=True)
    darkest_color = min(colors, key=lambda col: sum(col))
    return tuple(int(c) for c in darkest_color)

def add_text_below_logo(image, text, slogan, text_font_path, slogan_font_path):
    """
    Add text and slogan below the processed logo image.
    """
    cropped_logo_pil = process_logo_image(image)
    text_color = find_darkest_color(cropped_logo_pil)
    canvas_width, canvas_height = 400, 200  # Adjusted for potentially larger content
    new_canvas = Image.new('RGB', (canvas_width, canvas_height), "white")
    draw = ImageDraw.Draw(new_canvas)

    main_text_font = calculate_font_size(text, text_font_path, canvas_width - 40)
    slogan_font = calculate_font_size(slogan, slogan_font_path, canvas_width - 40)

    # Estimate text size for positioning
    main_text_width, main_text_height = main_text_font.getsize(text)
    slogan_width, slogan_height = slogan_font.getsize(slogan)

    # Positioning and pasting the logo
    logo_width, logo_height = cropped_logo_pil.size
    logo_x, logo_y = (canvas_width - logo_width) // 2, 30
    new_canvas.paste(cropped_logo_pil, (logo_x, logo_y))

    # Adding text below the logo
    draw.text(((canvas_width - main_text_width) // 2, logo_y + logo_height + 20), text, fill=text_color, font=main_text_font)
    draw.text(((canvas_width - slogan_width) // 2, logo_y + logo_height + 20 + main_text_height + 10), slogan, fill=text_color, font=slogan_font)

    return new_canvas


In [3]:
pip install rembg

  Obtaining dependency information for rembg from https://files.pythonhosted.org/packages/90/b0/8c8dbf4c7b45243aedfcc5234c5e94c59ade3a9e10b70a373f567361e5b6/rembg-2.0.54-py3-none-any.whl.metadata
  Using cached rembg-2.0.54-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/e0/05/3b50a005bf8dc68660d1a686a6c98ed4985844eee4d6156adc1667ca3c49/onnxruntime-1.17.0-cp311-cp311-win_amd64.whl.metadata
  Using cached onnxruntime-1.17.0-cp311-cp311-win_amd64.whl.metadata (4.3 kB)
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/20/44/458a0a135866f5e08266566b32ad9a182a7a059a894effe6c41a9c841ff1/opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for pooch from https://files.pythonhosted.org/packages/1a/a5/5174dac3957ac412e80a00f3

In [5]:
from rembg import remove 
from PIL import Image 
  
# Store path of the image in the variable input_path 
input_path =  "C:/Users/Abdul Wahab/ICMS Brochure/Generated_images/2024_01_18-03_02_29_PM.png"
  
# Store path of the output image in the variable output_path 
output_path = "C:/Users/Abdul Wahab/ICMS Brochure/Generated_images/gen_logo_bg.png"
  
# Processing the image 
input = Image.open(input_path) 
  
# Removing the background from the given Image 
output = remove(input) 
  
#Saving the image in the given path 
output.save(output_path) 

In [ ]:
# working code

In [ ]:
Flat vector logo of a curved wave, blue, trending on Dribble
Line art logo of a owl, golden, minimal, solid black background
Organic logo, shape of a leaf
Typographical logo, floral, letter” A”, serif typeface
Minimal logo of a cafe, a coffee bean, gradient brown color
Flat vector logo of a hummingbird, by Robb Janoff
Logo of a sneaker, trending color palette, by Leo Natsume, 3D illustration, pop up color, vibrant

In [ ]:
Bonus Tip: If you are not sure how to start, try this formula to begin with
Logo Design of [ Industry or Product ], [ Descriptive keywords and items ], [ Include 1–3 styles ], [ Color Palette ]